In [2]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [3]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-07 15:24:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.77MB/s    in 0.2s    

2023-03-07 15:24:12 (5.77 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url="https://my-pg-db.s3.amazonaws.com/amazon_reviews_us_Watches_v1_00.tsv" 
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_3/user_data.csv"
spark.sparkContext.addFile(url)
watches_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
watches_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [9]:
# Get the number of rows in the DataFrame.
watches_data_df.count()

960872

# Transform the Data

## Create the "review_id_table".

In [10]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = watches_data_df.select(["review_id", "customer_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+--------------+-----------+
|     review_id|customer_id|product_parent|review_date|
+--------------+-----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|     937001370| 2015-08-31|
| RKH8BNC3L5DLF|   14661224|     484010722| 2015-08-31|
|R2HLE8WKZSU3NL|   27324930|     361166390| 2015-08-31|
|R31U3UH5AZ42LL|    7211452|     958035625| 2015-08-31|
|R2SV659OUJ945Y|   12733322|     765328221| 2015-08-31|
| RA51CP8TR5A2L|    6576411|     230493695| 2015-08-31|
| RB2Q7DLDN6TH6|   11811565|     549298279| 2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|     844009113| 2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|     263720892| 2015-08-31|
| RX27XIIWY5JPB|   44751341|     124278407| 2015-08-31|
|R15C7QEZT0LGZN|    9962330|      28017857| 2015-08-31|
|R361XSS37V0NCZ|   16097204|     685450910| 2015-08-31|
| ROTNLALUAJAUB|   51330346|     767769082| 2015-08-31|
|R2DYX7QU6BGOHR|    4201739|     648595227| 2015-08-31|
| RWASY7FKI7QOT|   26339765|     457338020| 2015

## Create the "products" Table

In [11]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = watches_data_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00FALQ1ZC|Invicta Women's 1...|
|B00D3RGO20|Kenneth Cole New ...|
|B00DKYC7TK|Ritche 22mm Black...|
|B000EQS1JW|Citizen Men's BM8...|
|B00A6GFD7S|Orient ER27009B M...|
|B00EYSOSE8|Casio Men's GW-94...|
|B00WM0QA3M|Fossil Women's ES...|
|B00A4EYBR0|INFANTRY Mens Nig...|
|B00MAMPGGE|G-Shock Men's Gre...|
|B004LBPB7Q|Heiden Quad Watch...|
|B00KGTVGKS|Fossil Women's ES...|
|B0039UT5OU|Casio General Men...|
|B00MPF0XJQ|2Tone Gold Silver...|
|B003P1OHHS|Bulova Men's 98B1...|
|B00R70YEOE|Casio - G-Shock -...|
|B000FVE3BG|Invicta Men's 332...|
|B008X6JB12|Seiko Women's SUT...|
|B0040UOFPW|Anne Klein Women'...|
|B00UR2R5UY|Guess U13630G1 Me...|
|B00HFF57L0|Nixon Men's Geo V...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [16]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = watches_data_df.groupby("customer_id").count()
customers_df = customers_df.withColumnRenamed('count','customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1567510|             1|
|   19502021|             1|
|   12819130|             1|
|   35329257|             2|
|     108460|             1|
|    5453476|             1|
|   29913055|             1|
|   30717305|             1|
|    1570030|             1|
|   19032020|             1|
|   44178035|             1|
|   26079415|             2|
|   14230926|             1|
|   43478048|             2|
|   43694941|             1|
|   12318815|             3|
|   13731855|             1|
|     740134|             1|
|   41956754|             1|
|   20324070|             3|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [17]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = watches_data_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
| RA51CP8TR5A2L|          5|            0|          0|   N|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|
| RX27XIIWY5JPB|          4|            0|          0|   N|
|R15C7QEZT0LGZN|          4|            2|          2|   N|
|R361XSS37V0NCZ|          1|            0|          0|   N|
| ROTNLALUAJAUB|          3|            0|          0|   N|
|R2DYX7QU6BGOHR|          5|            

# Load

In [18]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"root", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [20]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [22]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [23]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [24]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)